In [1]:
import sys
sys.path.append('../')

import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100

%matplotlib inline

radiant_cols = ['hero_' + str(i) for i in range(5)]
dire_cols = ['hero_' + str(i) for i in range(5, 10)]

unique_roles = [
    'Carry',
    'Disabler',
    'Durable',
    'Escape',
    'Initiator',
    'Jungler',
    'Nuker',
    'Pusher',
    'Support'
]

unique_primary_attrs = [
    'agi',
    'int',
    'str'
]

In [2]:
df = pd.read_csv('../data/matches_data.csv')
df.head()

,match_id,hero_0,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,hero_8,hero_9,radiant_win
0,4154000815,83,64,93,99,22,110,32,41,92,11,False
1,4152678603,17,26,99,48,9,70,27,2,1,75,True
2,4152494309,23,120,82,18,26,108,31,7,14,44,False
3,4152425012,121,18,106,21,86,10,82,112,110,14,False
4,4152417604,77,93,45,26,7,119,14,4,106,70,False


In [3]:
cols_to_drop = [str(i) + '_pick' for i in range(1, 9)] + [str(i) + '_win' for i in range(1, 9)]
cols_to_drop += [
    'cm_enabled',  # uninformative
    'hero_id',  # uninformative
    'icon',  # uninformative
    'img',  # uninformative
    'localized_name',  # uninformative
    'name',  # uninformative
    'base_mana',  # nunique = 1
    'base_mana_regen',  # nunique = 1
    'base_health',  # nunique = 1
]

In [4]:
with open('../data/hero_stats.json', 'r') as f:
    heroes_stats = json.load(f)
    heroes_stats = pd.DataFrame(heroes_stats).set_index('id')
    
no_heroes = len(heroes_stats)

heroes_stats = heroes_stats.drop(cols_to_drop, axis=1)

heroes_stats.head()

,agi_gain,attack_range,attack_rate,attack_type,base_agi,base_armor,base_attack_max,base_attack_min,base_health_regen,base_int,base_mr,base_str,int_gain,legs,move_speed,primary_attr,pro_ban,pro_pick,pro_win,projectile_speed,roles,str_gain,turn_rate
id,,,,,,,,,,,,,,,,,,,,,,,
1,2.8,150,1.4,Melee,22,-1.0,33,29,1.75,12,25,23,1.8,2,310,agi,20.0,17.0,10.0,0,"[Carry, Escape, Nuker]",1.3,0.5
2,2.2,150,1.7,Melee,20,-2.0,28,24,4.25,18,25,25,1.6,2,290,str,43.0,57.0,33.0,900,"[Initiator, Durable, Disabler, Jungler]",2.8,0.6
3,2.4,400,1.7,Ranged,23,1.0,41,35,1.50,23,25,23,2.4,4,310,int,48.0,87.0,44.0,900,"[Support, Disabler, Nuker, Durable]",2.4,0.6
4,3.0,150,1.7,Melee,24,0.0,39,33,1.50,18,25,24,1.7,2,285,agi,89.0,53.0,26.0,900,"[Carry, Disabler, Jungler, Nuker, Initiator]",2.7,0.5
5,1.6,600,1.7,Ranged,16,0.0,32,26,1.50,14,25,18,2.9,2,275,int,5.0,67.0,41.0,900,"[Support, Disabler, Nuker, Jungler]",2.0,0.5


In [5]:
heroes_stats.drop('roles', axis=1).nunique()

agi_gain             33
attack_range         25
attack_rate           9
attack_type           2
base_agi             20
base_armor           11
base_attack_max      35
base_attack_min      33
base_health_regen     8
base_int             17
base_mr               2
base_str             17
int_gain             33
legs                  5
move_speed           14
primary_attr          3
pro_ban              66
pro_pick             69
pro_win              55
projectile_speed     13
str_gain             25
turn_rate             7
dtype: int64

In [6]:
cont_variables = [
    'agi_gain',
    'attack_range',
    'attack_rate',
    'base_agi',
    'base_armor',
    'base_attack_max',
    'base_attack_min',
    'base_health_regen',
    'base_int',
    'base_mr',
    'base_str',
    'int_gain',
    'legs',
    'move_speed',
    'pro_ban',
    'pro_pick',
    'pro_win',
    'projectile_speed',
    'str_gain',
    'turn_rate',
]

cat_variables = [
    'attack_type',
    'primary_attr',
    'roles',
]

In [7]:
def features(row, is_radiant=True):
    
    features = {col: 0 for col in cont_variables}
    features.update({
        'no_agi': 0,
        'no_int': 0,
        'no_str': 0,
        'no_melees': 0,
        'no_Carry': 0,
        'no_Disabler': 0,
        'no_Durable': 0,
        'no_Escape': 0,
        'no_Initiator': 0,
        'no_Jungler': 0,
        'no_Nuker': 0,
        'no_Pusher': 0,
        'no_Support': 0,
    })
    
    hero_cols = radiant_cols if is_radiant else dire_cols
    
    for i, hero in row[hero_cols].iteritems():
        
        hero_row = heroes_stats.loc[hero]
        
        for col in cont_variables:
            features[col] += hero_row[col]
            
        if hero_row['attack_type'] == 'Melee':
            features['no_melees'] += 1
            
        for attr in unique_primary_attrs:
            if hero_row['primary_attr'] == attr:
                features['no_{}'.format(attr)] += 1
            
        for role in unique_roles:
            if role in hero_row['roles']:
                features['no_{}'.format(role)] += 1
            
    return pd.Series(features)

In [8]:
radiant_features = df.apply(lambda x: features(x), axis=1)
dire_features = df.apply(lambda x: features(x, is_radiant=False), axis=1)

In [9]:
radiant_features.head()

,agi_gain,attack_range,attack_rate,base_agi,base_armor,base_attack_max,base_attack_min,base_health_regen,base_int,base_mr,base_str,int_gain,legs,move_speed,pro_ban,pro_pick,pro_win,projectile_speed,str_gain,turn_rate,no_agi,no_int,no_str,no_melees,no_Carry,no_Disabler,no_Durable,no_Escape,no_Initiator,no_Jungler,no_Nuker,no_Pusher,no_Support
0,7.7,1230.0,8.8,74.0,-2.0,219.0,177.0,9.25,96.0,125.0,116.0,11.8,10.0,1435.0,295.0,211.0,108.0,4000.0,12.40,3.20,1.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,0.0,4.0,1.0,2.0
1,11.3,2190.0,8.6,90.0,0.0,157.0,120.0,7.50,87.0,125.0,95.0,12.3,10.0,1485.0,238.0,315.0,160.0,4700.0,10.35,3.40,2.0,2.0,1.0,1.0,4.0,3.0,1.0,2.0,3.0,0.0,5.0,1.0,2.0
2,9.8,1200.0,8.6,91.0,5.0,177.0,147.0,8.00,88.0,125.0,106.0,9.3,10.0,1485.0,148.0,103.0,56.0,3600.0,12.40,3.35,2.0,1.0,2.0,4.0,4.0,5.0,3.0,2.0,5.0,0.0,5.0,1.0,2.0
3,8.7,2000.0,8.4,92.0,-2.0,175.0,143.0,8.00,102.0,125.0,102.0,11.4,10.0,1470.0,NaN,NaN,NaN,4175.0,12.20,3.20,1.0,3.0,1.0,2.0,3.0,5.0,1.0,3.0,2.0,0.0,5.0,0.0,3.0
4,6.4,1680.0,8.5,80.0,-3.0,175.0,138.0,10.25,91.0,125.0,105.0,12.4,10.0,1530.0,347.0,374.0,195.0,2700.0,12.20,3.00,1.0,2.0,2.0,3.0,2.0,3.0,1.0,2.0,2.0,1.0,4.0,2.0,2.0


In [10]:
dire_features.head()

,agi_gain,attack_range,attack_rate,base_agi,base_armor,base_attack_max,base_attack_min,base_health_regen,base_int,base_mr,base_str,int_gain,legs,move_speed,pro_ban,pro_pick,pro_win,projectile_speed,str_gain,turn_rate,no_agi,no_int,no_str,no_melees,no_Carry,no_Disabler,no_Durable,no_Escape,no_Initiator,no_Jungler,no_Nuker,no_Pusher,no_Support
0,10.50,1900.0,8.5,96.0,-5.0,144.0,108.0,9.75,87.0,110.0,101.0,9.10,8.0,1445.0,219.0,231.0,125.0,5000.0,12.5,4.10,3.0,1.0,1.0,2.0,3.0,4.0,2.0,3.0,2.0,0.0,3.0,1.0,2.0
1,11.70,1450.0,8.2,98.0,2.0,180.0,147.0,11.00,92.0,125.0,114.0,10.40,10.0,1485.0,447.0,311.0,151.0,3700.0,12.0,2.70,2.0,2.0,1.0,3.0,3.0,4.0,2.0,1.0,3.0,2.0,3.0,1.0,2.0
2,9.35,1150.0,8.5,76.0,0.0,180.0,147.0,8.75,78.0,125.0,113.0,10.55,10.0,1490.0,196.0,313.0,159.0,2700.0,13.6,3.30,1.0,1.0,3.0,4.0,1.0,3.0,2.0,2.0,2.0,0.0,4.0,0.0,3.0
3,10.60,1575.0,8.3,89.0,-4.0,139.0,101.0,7.75,91.0,125.0,114.0,9.10,8.0,1430.0,385.0,377.0,199.0,4000.0,13.0,3.45,2.0,1.0,2.0,2.0,2.0,5.0,2.0,3.0,3.0,0.0,5.0,1.0,2.0
4,10.00,1075.0,8.1,96.0,1.0,176.0,148.0,8.50,86.0,125.0,113.0,9.50,10.0,1465.0,310.0,317.0,136.0,3900.0,13.7,2.90,3.0,1.0,1.0,4.0,3.0,5.0,2.0,2.0,3.0,2.0,4.0,0.0,1.0


In [11]:
radiant_features.nunique()

agi_gain              494
attack_range          407
attack_rate            55
base_agi               77
base_armor             53
base_attack_max       153
base_attack_min       142
base_health_regen      39
base_int               58
base_mr                 2
base_str               52
int_gain              438
legs                   12
move_speed             46
pro_ban              1014
pro_pick              676
pro_win               365
projectile_speed      198
str_gain              415
turn_rate              77
no_agi                  6
no_int                  6
no_str                  6
no_melees               6
no_Carry                6
no_Disabler             6
no_Durable              6
no_Escape               6
no_Initiator            6
no_Jungler              5
no_Nuker                6
no_Pusher               6
no_Support              6
dtype: int64

In [12]:
radiant_features.to_csv('../data/radiant_features.csv', index=False)
dire_features.to_csv('../data/dire_features.csv', index=False)